In [78]:
import json
from streamio import jsonstream
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
#run this once then restart your notebook
#make sure you have a folder named"test_data" wherein you have stored all the testfiles
import csv
import sys
reload(sys)
sys.setdefaultencoding('utf8')
test_data_files=['sample1_period1','sample2_period2','sample3_period3','sample4_period1','sample5_period1','sample6_period2','sample7_period3','sample8_period1','sample9_period2','sample10_period3']
for filename in test_data_files:
    csv_name='test_data/'+filename+'.csv'
    with open(csv_name, 'ab') as temp:
        writer = csv.DictWriter(temp, fieldnames = ['citation_date','author','retweetcount','follower_count','influence_level','ranking_score','replies','impressions','favorite_count','hour_of_the_day','tweet_count'])
        writer.writeheader()        
        temp.close()
    s='test_data/'+filename+'.txt'    
    with open(s) as f:
        for line in f:
            item=json.loads(line)
            with open(csv_name, 'ab') as myfile:
                wr = csv.writer(myfile)
                mylist=[item["citation_date"],item["author"]["name"],item["metrics"]["citations"]["total"],item["author"]["followers"],item.get('author').get('influence_level'),item['metrics']['ranking_score'],item.get('metrics').get('citations').get('replies'),item.get('metrics').get('impressions'),item['tweet']['favorite_count'],datetime.datetime.fromtimestamp(item["citation_date"]).hour,1]
                wr.writerow(mylist)
                myfile.close()

In [79]:
features=['hour_of_the_day','retweetcount','follower_count','influence_level','replies','ranking_score','impressions','favorite_count','tweet_count']
files=['gopatriots','gohawks','patriots','nfl','sb49','superbowl']
import datetime, time
import pytz
ptz = pytz.timezone('US/Pacific')

In [80]:
consolidated=[]
for f in files:
    filename=f+'.csv'
    n=pd.read_csv(filename)
    temp=n
    temp['citation_date']=n['citation_date']//60//60
    
    temp=temp.groupby(['citation_date','hour_of_the_day'])['retweetcount','follower_count','influence_level','replies','ranking_score','impressions','favorite_count','tweet_count'].sum()
    temp=temp.reset_index()
    consolidated.append(temp)
    
total=pd.concat(consolidated)
total=total.fillna(0)

In [86]:
temp=total.groupby(['citation_date','hour_of_the_day'])['retweetcount','follower_count','influence_level','replies','ranking_score','impressions','favorite_count','tweet_count'].mean()
temp=temp.reset_index()
yyy=temp[4:]

In [87]:
indexed=temp.set_index(['citation_date'])

In [88]:
#sliding window of 5 hours
final_output=indexed.rolling(5).mean()


In [90]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=70, random_state=0)
a=final_output[4:]

x_data=np.array(a.iloc[:,0:])
y_data=np.array(yyy.iloc[:,9])
x_data=np.nan_to_num(x_data)
x_data=x_data[:-1]
y_data=y_data[1:]
regr = RandomForestRegressor(n_estimators=70, random_state=0)
regr.fit(x_data,y_data)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [115]:
from sklearn.metrics import mean_absolute_error
test_data_files=['sample1_period1','sample2_period2','sample3_period3','sample4_period1','sample5_period1','sample6_period2','sample7_period3','sample9_period2','sample10_period3']
actual=[]
predicted_list=[]
for filename in test_data_files:
    csv_name='test_data/'+filename+'.csv'
    print csv_name
    n=pd.read_csv(csv_name)
    n.fillna(0)
    temp=n
    
    temp['citation_date']=n['citation_date']//60//60
    test=temp.groupby(['citation_date','hour_of_the_day'])['retweetcount','follower_count','influence_level','replies','ranking_score','impressions','favorite_count','tweet_count'].sum()
    test=test.reset_index()
    l=len(test)-1
    t=test[l-5:l]
    c=np.array(t.mean())[1:].reshape(1,-1)
    pred= regr.predict(c)
    actual.append(np.array(test[l:]['tweet_count'])[0])
    predicted_list.append(pred[0])
    print filename
    print "Predicted: ", pred
    print "Actual: ", np.array(test[l:]['tweet_count'])[0]

test_data/sample1_period1.csv
sample1_period1
Predicted:  [ 84.24333333]
Actual:  1
test_data/sample2_period2.csv
sample2_period2
Predicted:  [ 25.32095238]
Actual:  4
test_data/sample3_period3.csv
sample3_period3
Predicted:  [ 1172.27238095]
Actual:  523
test_data/sample4_period1.csv
sample4_period1
Predicted:  [ 184.39428571]
Actual:  201
test_data/sample5_period1.csv
sample5_period1
Predicted:  [ 132.19714286]
Actual:  1
test_data/sample6_period2.csv
sample6_period2
Predicted:  [ 31.60190476]
Actual:  14
test_data/sample7_period3.csv
sample7_period3
Predicted:  [ 71.05761905]
Actual:  120
test_data/sample9_period2.csv
sample9_period2
Predicted:  [ 1621.61714286]
Actual:  1
test_data/sample10_period3.csv
sample10_period3
Predicted:  [ 37.68571429]
Actual:  61


In [116]:
print actual
print predicted_list

[1, 4, 523, 201, 1, 14, 120, 1, 61]
[84.243333333333354, 25.320952380952384, 1172.2723809523809, 184.39428571428576, 132.19714285714281, 31.601904761904773, 71.057619047619042, 1621.6171428571424, 37.685714285714283]
